In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 7-2. ブロードキャスティングの実装

### 7-2-1. ドット構文も構文糖衣

#### コード7-15. ドット構文と等価な関数呼び出し例(1)

In [2]:
sin.([0.0, 1.0, π/2])

3-element Vector{Float64}:
 0.0
 0.8414709848078965
 1.0

In [3]:
broadcast(sin, [0.0, 1.0, π/2])

3-element Vector{Float64}:
 0.0
 0.8414709848078965
 1.0

In [4]:
(1:9) .+ (1:9)'

9×9 Matrix{Int64}:
  2   3   4   5   6   7   8   9  10
  3   4   5   6   7   8   9  10  11
  4   5   6   7   8   9  10  11  12
  5   6   7   8   9  10  11  12  13
  6   7   8   9  10  11  12  13  14
  7   8   9  10  11  12  13  14  15
  8   9  10  11  12  13  14  15  16
  9  10  11  12  13  14  15  16  17
 10  11  12  13  14  15  16  17  18

In [5]:
broadcast(+, (1:9), (1:9)')

9×9 Matrix{Int64}:
  2   3   4   5   6   7   8   9  10
  3   4   5   6   7   8   9  10  11
  4   5   6   7   8   9  10  11  12
  5   6   7   8   9  10  11  12  13
  6   7   8   9  10  11  12  13  14
  7   8   9  10  11  12  13  14  15
  8   9  10  11  12  13  14  15  16
  9  10  11  12  13  14  15  16  17
 10  11  12  13  14  15  16  17  18

#### コード7-16. ドット構文と等価な関数呼び出し例(2)

In [6]:
A = Matrix{Float32}(undef, 3, 3);

In [7]:
A .= sqrt.([1, 4, 9]);
A

3×3 Matrix{Float32}:
 1.0  1.0  1.0
 2.0  2.0  2.0
 3.0  3.0  3.0

In [8]:
A .+= [.1 .2 .3];
A

3×3 Matrix{Float32}:
 1.1  1.2  1.3
 2.1  2.2  2.3
 3.1  3.2  3.3

In [9]:
broadcast!(sqrt, A, [1, 4, 9]);
A

3×3 Matrix{Float32}:
 1.0  1.0  1.0
 2.0  2.0  2.0
 3.0  3.0  3.0

In [10]:
broadcast!(+, A, A, [.1 .2 .3]);
A

3×3 Matrix{Float32}:
 1.1  1.2  1.3
 2.1  2.2  2.3
 3.1  3.2  3.3

### 7-2-2. ブロードキャスティングの仕組み

#### 仮想コード7-1. broadcast()/broadcast!() 関数と等価なコード

```julia
# equivalent to `broadcast(fn, xs...)`
Broadcast.materialize(Broadcast.broadcasted(f, xs...))

# equivalent to `broadcast!(fn, dest, xs...)`
Broadcast.materialize!(dest, Broadcast.broadcasted(f, xs...))
```

#### コード7-17. ドット構文とデシュガーされたコード例(1)

In [11]:
sin.([0.0, 1.0, π/2])

3-element Vector{Float64}:
 0.0
 0.8414709848078965
 1.0

In [12]:
Broadcast.materialize(Broadcast.broadcasted(sin, [0.0, 1.0, π/2]))

3-element Vector{Float64}:
 0.0
 0.8414709848078965
 1.0

In [13]:
(1:9) .+ (1:9)'

9×9 Matrix{Int64}:
  2   3   4   5   6   7   8   9  10
  3   4   5   6   7   8   9  10  11
  4   5   6   7   8   9  10  11  12
  5   6   7   8   9  10  11  12  13
  6   7   8   9  10  11  12  13  14
  7   8   9  10  11  12  13  14  15
  8   9  10  11  12  13  14  15  16
  9  10  11  12  13  14  15  16  17
 10  11  12  13  14  15  16  17  18

In [14]:
Broadcast.materialize(Broadcast.broadcasted(+, (1:9), (1:9)'))

9×9 Matrix{Int64}:
  2   3   4   5   6   7   8   9  10
  3   4   5   6   7   8   9  10  11
  4   5   6   7   8   9  10  11  12
  5   6   7   8   9  10  11  12  13
  6   7   8   9  10  11  12  13  14
  7   8   9  10  11  12  13  14  15
  8   9  10  11  12  13  14  15  16
  9  10  11  12  13  14  15  16  17
 10  11  12  13  14  15  16  17  18

#### コード7-18. ドット構文とデシュガーされたコード例(2)

In [15]:
A = Matrix{Float32}(undef, 3, 3);

In [16]:
A .= sqrt.([1, 4, 9]);
A

3×3 Matrix{Float32}:
 1.0  1.0  1.0
 2.0  2.0  2.0
 3.0  3.0  3.0

In [17]:
A .+= [.1 .2 .3];
A

3×3 Matrix{Float32}:
 1.1  1.2  1.3
 2.1  2.2  2.3
 3.1  3.2  3.3

In [18]:
Broadcast.materialize!(A, Broadcast.broadcasted(sqrt, [1, 4, 9]));
A

3×3 Matrix{Float32}:
 1.0  1.0  1.0
 2.0  2.0  2.0
 3.0  3.0  3.0

In [19]:
Broadcast.materialize!(A, Broadcast.broadcasted(+, A, [.1 .2 .3]));
A

3×3 Matrix{Float32}:
 1.1  1.2  1.3
 2.1  2.2  2.3
 3.1  3.2  3.3

#### `Broadcast.broadcasted()` と `Broadcast.materialize()`

##### コード7-19. Broadcast.broadcastable() の動作例

In [20]:
Broadcast.broadcastable([1, 2, 3])  # そのまま返す

3-element Vector{Int64}:
 1
 2
 3

In [21]:
Broadcast.broadcastable([1. 2.; 3. 4.])  # そのまま返す

2×2 Matrix{Float64}:
 1.0  2.0
 3.0  4.0

In [22]:
Broadcast.broadcastable(('a', :ok, π))  # そのまま返す

('a', :ok, π)

In [23]:
Broadcast.broadcastable(1)

1

In [24]:
Broadcast.broadcastable("string")

Base.RefValue{String}("string")

In [25]:
Broadcast.broadcastable(Int)

Base.RefValue{Type{Int64}}(Int64)

In [26]:
Broadcast.broadcastable(Set([1, 2, 3]))  # collect(Set([1, 2, 3])) と一致

3-element Vector{Int64}:
 2
 3
 1

In [27]:
Broadcast.broadcastable(Dict(:a=>1, :b=>2))

LoadError: ArgumentError: broadcasting over dictionaries and `NamedTuple`s is reserved

In [28]:
Broadcast.broadcastable((a=1, b="B"))

LoadError: ArgumentError: broadcasting over dictionaries and `NamedTuple`s is reserved

In [29]:
struct MySingleton end
Broadcast.broadcastable(MySingleton())

LoadError: MethodError: no method matching length(::MySingleton)
[0mClosest candidates are:
[0m  length([91m::Union{Base.KeySet, Base.ValueIterator}[39m) at abstractdict.jl:58
[0m  length([91m::Union{LinearAlgebra.Adjoint{T, S}, LinearAlgebra.Transpose{T, S}} where {T, S}[39m) at ~/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/adjtrans.jl:172
[0m  length([91m::Union{ZMQ._Message, Base.RefValue{ZMQ._Message}}[39m) at ~/.julia/packages/ZMQ/lrABE/src/_message.jl:31
[0m  ...

##### コード7-20. ブロードキャスティングの例(6)

In [30]:
A = [1 1; 1 0];

In [31]:
A .^ (0:10)

LoadError: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 2 and 11

In [32]:
Ref(A) .^ (0:10)

11-element Vector{Matrix{Int64}}:
 [1 0; 0 1]
 [1 1; 1 0]
 [2 1; 1 1]
 [3 2; 2 1]
 [5 3; 3 2]
 [8 5; 5 3]
 [13 8; 8 5]
 [21 13; 13 8]
 [34 21; 21 13]
 [55 34; 34 21]
 [89 55; 55 34]

In [33]:
replace_with_dict(str::AbstractString, dict::AbstractDict) = 
    replace(str, ∈(keys(dict))=>c->dict[c])

replace_with_dict (generic function with 1 method)

In [34]:
dict0 = Dict('o'=>'0', 'i'=>'1', 'z'=>'2', 'e'=>'3',
    'a'=>'4', 's'=>'5', 'b'=>'6', 't'=>'7');

In [35]:
replace_with_dict("Julia", dict0)

"Jul14"

In [36]:
strs = ["Julia", "Python", "Ruby", "Haskell"];

In [37]:
replace_with_dict.(strs, Ref(dict0))

4-element Vector{String}:
 "Jul14"
 "Py7h0n"
 "Ru6y"
 "H45k3ll"

In [38]:
replace_with_dict.(strs, dict0)

LoadError: ArgumentError: broadcasting over dictionaries and `NamedTuple`s is reserved

#### `Broadcast.BroadcastStyle`

##### コード7-21. `Broadcast.BroadcastStyle` の確認(1)

In [39]:
Broadcast.broadcastable([1, 2, 3]) |> typeof |> Broadcast.BroadcastStyle

Base.Broadcast.DefaultArrayStyle{1}()

In [40]:
Broadcast.broadcastable(Float32[1 2; 3 4]) |> typeof |> Broadcast.BroadcastStyle

Base.Broadcast.DefaultArrayStyle{2}()

In [41]:
Broadcast.broadcastable(1:10) |> typeof |> Broadcast.BroadcastStyle

Base.Broadcast.DefaultArrayStyle{1}()

In [42]:
Broadcast.broadcastable((1, 2.0, 3//1)) |> typeof |> Broadcast.BroadcastStyle

Base.Broadcast.Style{Tuple}()

In [43]:
Broadcast.broadcastable(1) |> typeof |> Broadcast.BroadcastStyle

Base.Broadcast.DefaultArrayStyle{0}()

In [44]:
Broadcast.broadcastable("string") |> typeof |> Broadcast.BroadcastStyle

Base.Broadcast.DefaultArrayStyle{0}()

In [45]:
Broadcast.broadcastable(Ref([1 1; 1 0])) |> typeof |> Broadcast.BroadcastStyle

Base.Broadcast.DefaultArrayStyle{0}()

##### コード7-22. `Broadcast.BroadcastStyle` の確認(2)

In [46]:
bs0 = Broadcast.BroadcastStyle(Int)

Base.Broadcast.DefaultArrayStyle{0}()

In [47]:
bs1 = Broadcast.BroadcastStyle(Vector{Int})

Base.Broadcast.DefaultArrayStyle{1}()

In [48]:
bs2 = Broadcast.BroadcastStyle(Matrix{Int})

Base.Broadcast.DefaultArrayStyle{2}()

In [49]:
bst = Broadcast.BroadcastStyle(typeof((1, 2, 3)))

Base.Broadcast.Style{Tuple}()

In [50]:
Broadcast.BroadcastStyle(bs0, bs1)

Base.Broadcast.DefaultArrayStyle{1}()

In [51]:
Broadcast.BroadcastStyle(bs0, bs2)

Base.Broadcast.DefaultArrayStyle{2}()

In [52]:
Broadcast.BroadcastStyle(bs1, bs2)

Base.Broadcast.DefaultArrayStyle{2}()

In [53]:
Broadcast.BroadcastStyle(bs0, bst)

Base.Broadcast.Style{Tuple}()

In [54]:
Broadcast.BroadcastStyle(bs1, bst)

Base.Broadcast.DefaultArrayStyle{1}()

In [55]:
Broadcast.BroadcastStyle(bs2, bst)

Base.Broadcast.DefaultArrayStyle{2}()

In [56]:
Broadcast.BroadcastStyle(bs1, bs0)

Base.Broadcast.DefaultArrayStyle{1}()

In [57]:
Broadcast.BroadcastStyle(bst, bs0)

Base.Broadcast.Unknown()

In [58]:
all(Broadcast.BroadcastStyle(bs, bs) === bs for bs in (bs0, bs1, bs2, bst))

true